<h1 style="color:#5A8E00; font-weight: bold;">
Algorithme A\*
</h1>
<br>Nous allons de ce pas nous immerger dans une application de l'algorithme A\*. Le but du script qui suit va être de déterminer les affectations qui minimisent le coût total d'une opération à partir d'un tableau à 2 dimensions dans lequel sont représentés les-dits coûts.

On importe numpy pour réaliser des opérations plus vite que l'éclair, pandas pour afficher les résultats de manière élégante, ipywidgets pour intéragir de manière agréable avec ce notebook et heapq qui n'est autre qu'une implémentation des Tas (qui vont nous permettre de gagner un peu / pas mal de temps de calcul).
<br>Un petit coup d'oeil sur la documentation de heapq et notamment des fonctions <b style="color:#DB5B30">heappush</b> et <b style="color:#DB5B30">heappop</b> ne fait pas de mal : https://docs.python.org/3/library/heapq.html

In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from heapq import heappush, heappop

Ci-dessous sont les différentes heuristiques que nous avons trouvées. Chacune des fonctions pour le calcul de h prend donc en paramètre le noeud dont on doit calculer h et la table des coûts.
<br>Chacune des fonctions commence de la même manière. Tout d'abord si le noeud est un <b style="color:#DB5B30">but</b> alors <b style="color:#DB5B30">h=0</b> (car le cout du chemin optimal d'un noeud à lui même est de 0). Ensuite on isole sur la table des coûts les possible chemins restant à parcourir.
<br><b style="color:#DB5B30">Par exemple</b> si la matrice de coût est :
<table style="margin-left:0;font-family:'Arial'">
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">2</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">8</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">13</td>
  </tr>
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">15</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">2</td>
  </tr>
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">11</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">9</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">13</td>
  </tr>
</table>
<br>Et que nous sommes dans un noeud de profondeur 1 qui a eu comme première affectation 2 alors on garde seulement les lignes suivantes et on supprime toutes les colonnes qui sont déjà affectées, dans ce cas-ci seulement la colonne numéro 2.
<table style="margin-left:0;font-family:'Arial'">
  <tr style="background-color:white">
    <td style="background-color:#B7B7B7; border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">2</td>
    <td style="background-color:#B7B7B7; border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">8</td>
    <td style="background-color:#B7B7B7; border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">13</td>
  </tr>
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">15</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
    <td style="background-color:#B7B7B7; border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">2</td>
  </tr>
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">11</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">9</td>
    <td style="background-color:#B7B7B7; border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">13</td>
  </tr>
</table>
<br>Il nous reste donc la table suivante :
<table style="margin-left:0;font-family:'Arial'">
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">15</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
  </tr>
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">11</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">9</td>
  </tr>
</table>
<br>C'est ici que les heuristiques diffèrent. La première, somme_des_min_des_lignes, calcule comme son nom l'indique la somme des minimaux de chaque ligne. Dans notre exemple :
<table style="margin-left:0;font-family:'Arial'">
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">15</td>
    <td style="background-color:#FFA468; border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
  </tr>
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">11</td>
    <td style="background-color:#FFA468; border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">9</td>
  </tr>
</table>
<span style="background-color:#D5BAD6">h = 1 + 9 = 10</span>
<br><br>La seconde heuristique, somme_des_min_des_colonnes, calcule la somme des minimaux de chaque colonne. Dans notre exemple :
<table style="margin-left:0;font-family:'Arial'">
  <tr style="background-color:white">
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">15</td>
    <td style="background-color:#FFA468; border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">1</td>
  </tr>
  <tr style="background-color:white">
    <td style="background-color:#FFA468; border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">11</td>
    <td style="border: 1px solid black; border-collapse: collapse; width:30px; height:30px; text-align:center;">9</td>
  </tr>
</table>
<span style="background-color:#D5BAD6">h = 11 + 1 = 12</span>
<br><br>La troisème et dernière heuristique calcul tout simplement le maximum de ces deux valeurs, c'est à dire dans ce cas-ci :
<br><span style="background-color:#D5BAD6">h = max(10, 12) = 12</span>

<b style="color:#DB5B30">NB</b> : *np.delete(x, [0, 8], 1)* va supprimer du tableau x les indices 0 et 8 de l'axe 1 du tableau l'axe 1 représente les colonnes tu tableau. Un petit coup d'oeil vers la doc : https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.delete.html

*np.delete(couts[noeud.p:], noeud.affectations, 1)* renvoie donc un tableau qui supprime les colonnes déjà affectés du tableau de couts[p:] on a donc un tableau de couts sans les colonnes déjà affectés et avec les lignes qui restent à affecter.

<b style="color:#DB5B30">NB 2</b> : *np.amin(x, 1)* renvoie la liste des minimum des lignes de x. Doc : https://docs.scipy.org/doc/numpy/reference/generated/numpy.amin.html

In [ ]:
def somme_des_min_des_lignes(noeud, couts):
    if noeud.estBut():
        return 0
    temp = couts[noeud.p:][:, noeud.non_affectations]
    return np.amin(temp, 1).sum()

def somme_des_min_des_colonnes(noeud, couts):
    if noeud.estBut():
        return 0
    temp = couts[noeud.p:][:, noeud.non_affectations]
    return np.amin(temp, 0).sum()

def max_des_deux(noeud, couts):
    if noeud.estBut():
        return 0
    temp = couts[noeud.p:][:, noeud.non_affectations]
    return max(np.amin(temp, 1).sum(), np.amin(temp, 0).sum())

calculer_h = somme_des_min_des_lignes

Nous vous proposons ci-dessous de choisir laquelle des heuristiques utiliser pour l'algorithme (et ceci avec une interface sympatoche et dynamique). Toutefois si l'interface ne marche pas (ce qui peut arriver), il vous faudra affecter manuellement la fonction <b style="color:#DB5B30">calculer_h</b> et ce en rentrant dans une cellule :
<br><span style="color:blue">set_heuristique</span>(<span style="color:#BA2121">'Somme des min des lignes'</span> OU <span style="color:#BA2121">'Somme des min des colonnes'</span> OU <span style="color:#BA2121">'Max des deux'</span>)
<br><br>Mais à priori l'interface devrait fonctionner... Rien de bien intéressant dans la cellule ci-dessous du coup (mais à exécuter pour faire le choix de l'heuristique).

In [ ]:
def set_heuristique(heuristique):
    global calculer_h
    if heuristique == 'Somme des min des lignes':
        calculer_h = somme_des_min_des_lignes
    elif heuristique == 'Somme des min des colonnes':
        calculer_h = somme_des_min_des_colonnes
    else:
        calculer_h = max_des_deux

w = widgets.Dropdown(
    options=['Somme des min des lignes', 'Somme des min des colonnes', 'Max des deux'],
    description="Choix de l'heuristique :")
try:
    w.layout = widgets.Layout(width='375px')
    w.style.description_width = '150px'
except:
    pass
widgets.interact(set_heuristique, heuristique=w);

La classe Noeud ci-après représente une bonne partie de l'algorithme et mérite quelques éclaircissements. Nous vons invitons tout d'abord à jeter un oeil à cette dernière afin d'en notifier les principaux attributs et fonctions.

<u style="color:#5A8E00; font-size:16px">Attributs :</u>
* <b style="color:#DB5B30">n :</b> La profondeur maximale du graphe. Si le tableau de coûts est de dimensions m\*m alors n=m
* <b style="color:#DB5B30">g :</b> Le coût d'un chemin optimal du premier noeud à celui-ci (du noeud de profondeur 0 au noeud de profondeur p)
* <b style="color:#DB5B30">p :</b> La profondeur du noeud, c'est à dire le nombre d'ancêtres que possède ce noeud. Par exemple le premier noeud est de profondeur 0, et un noeud but est de profondeur n.
* <b style="color:#DB5B30">affectations :</b> Une liste contenant toutes les affectations "qui sont faîtes" pour ce noeud. Pour un noeud but on a une liste d'affectations complète qui décrira une situation d'allocations finie. Notre algorithme devant se terminer toujours par des affectations optimales, le dernier noeud choisi possèdera donc la liste des allocations optimales.
* <b style="color:#DB5B30">h :</b> L'heuristique du noeud. C'est une estimation du coût chemin optimal de ce noeud (de profondeur p) à un but (de profondeur n). Comme nous voulons trouver le chemin optimal de 0 à n, nous devons respecter la propriété $h(p) \leq h*(p)$ où h*(p) est le coût du chemin optimal de ce noeud à un but.
* <b style="color:#DB5B30">f :</b> Estimation du coût d'un chemin optimal de 0 à un but en passant par ce noeud. C'est donc ce paramètre là que l'on cherche à minimiser. Quand l'algorithme se termine, on veut que le noeud but sur lequel il se termine soit celui avec le plus petit f possible. Ce paramètre va donc nous servir à ordonner les noeuds.

<u style="color:#5A8E00; font-size:16px">Fonctions :</u>
* <b style="color:#DB5B30">estBut(self) :</b> Comme son nom l'indique cette fonction renvoie un booléen : True si le noeud est un but, False sinon. On exploite ici le fait que si le noeud est un but, alors sa profondeur est égale à la profondeur maximale du GRP.
* <b style="color:#DB5B30">developpe(self, calculer_h, couts) :</b> Cette fonction renvoie une liste de noeud développés à partir de celui sur lequel la fonction est appelée. Dans cette fonction on parcours tout simplement toutes les affectations possibles de faire depuis ce noeud et on crée les noeuds qui correspondent à toutes ces affecations.
* <b style="color:#DB5B30">\__lt__(self, other) :</b> va permettre de "classer" les objets Noeud (leur donne un ordre) ce qui va être très pratique pour l'implémentation du Tas. Un Noeud est "plus petit" qu'un autre si son **f** est plus petit que celui de l'autre.

In [ ]:
class Noeud:
    def __init__(self, n, couts, non_affectations, g=0, p=0, affectations=list()):
        self.n = n
        self.g = g
        self.p = p
        self.affectations = affectations
        self.non_affectations = non_affectations
        self.h = calculer_h(self, couts)
        self.f = self.g + self.h
    
    def estBut(self):
        return self.p == self.n
    
    def developpe(self, couts):
        heritiers = list()
        
        for indice_non_affectations, affectation in enumerate(self.non_affectations):
            heritiers.append(Noeud(self.n, couts, np.delete(self.non_affectations, indice_non_affectations),
                                   g=self.g+couts[self.p, affectation], p=self.p+1,
                                   affectations=self.affectations+[affectation]))
        
        return heritiers
    
    def __lt__(self, other):
        return self.f < other.f

<h3 style="color:#5A8E00; font-weight: bold;">Enfin, la progression !</h3>
<br>Nous avons suivi l'algorithme proposé par Nils John Nilsson. Cependant nous avons choisi un Tas plutôt qu'une liste pour la représentation d'OUVERT, en effet à chaque étape de la boucle, avec une liste, nous devions chercher le plus petit élément d'OUVERT ce qui était donc en O(k), k étant le nombre d'éléments dans OUVERT et pouvant rapidemment atteindre un nombre assez grand. Avec un Tas, il nous suffit d'ajouter chaque Noeud à ouvert en complexité log(k). Et on a à chaque étape de la boucle au plus n Noeud créés, c'est donc en O(n*log(k)) avec n << k.

Il est à noter que nous n'implémentons pas la liste FERME car notre GRP est un graphe sans cycle.

1. Tout premièrement créons une matrice <b style="color:#DB5B30">coût</b> aléatoirement.
2. Puis posons n = la longeur de coût, c'est la profondeur maximale du GRP.
3. Comme indiqué par M. Nilsson, mettons dans un Tas appelée OUVERT le sommet de départ du GRP
4. Ensuite bouclons tant que OUVERT n'est pas vide.
5. On prend le premier élément du Tas (le plut petit) et on l'en enlève (fonction <b style="color:#DB5B30">heappop</b>).
6. Si c'est un but on termine la procèdure avec succès.
7. On développe le noeud précédemment choisi et on met les noeuds résultants de cette opération dans OUVERT.

In [ ]:
couts = np.random.randint(1000, size=(20, 20))

n = len(couts)
non_affectations = np.arange(n)
OUVERT = [Noeud(n, couts, non_affectations)]

while len(OUVERT) > 0:
    
    noeud = heappop(OUVERT)
    
    if noeud.estBut():
        break
    
    for heritier in noeud.developpe(couts):
        heappush(OUVERT, heritier)

On a donc à la fin de l'exécution de l'algorithme l'ensemble des affectations dans noeud.affectations

In [ ]:
print(noeud.affectations)

Cette représentation étant assez peu lisible nous avons codé une fonction <span style="color:blue">print_affectations</span> qui affiche les résultats sous une forme plus esthétique.

In [ ]:
def print_affectations(couts, affectations):
    df = pd.DataFrame(couts)
    return df.style.apply(lambda tab: color_couts(len(couts), affectations), axis=None)

def color_couts(n, affectations):
    ar = [['']*n for i in range(n)]
    for i, j in enumerate(affectations):
        ar[i][j] = 'background-color: #FFC900'
    return pd.DataFrame(ar)

print_affectations(couts, noeud.affectations)

<h3 style="color:#5A8E00; font-weight: bold;">Meilleure heuristique</h3>
<br>On est bien content d'avoir trouvé nos affectations optimales, mais encore nous faut-il déterminer quelle est l'heuristique la plus efficace. Pour cela nous allons tout simplement regarder le nombre de noeuds développés par chacune des heuristiques pour les mêmes tables de coûts. On pourra ensuite comparer ce nombre de noeuds au nombre de noeuds total que possède le GRP.
<br><br>La fonction <span style="color:blue">tailleGRP</span>(n) calcul le nombre de noeuds total d'un GRP de profondeur maximale n.

In [ ]:
def tailleGRP(n):
    T = 1
    R = n
    for i in range(n, 1, -1):
        T += R
        R = R*(i-1)
    T += R
    return T

Maintenant créons la fonction <span style="color:blue">nb_noeuds_developpes</span>(couts, calculer_h) qui va renvoyer le nombre de noeuds développés pour la table <b style="color:#DB5B30">couts</b> avec la fonction de calcul heuristique <b style="color:#DB5B30">calculer_h</b>.

In [ ]:
def nb_noeuds_developpes(couts):
    n = len(couts)
    non_affectations = np.arange(n)
    OUVERT = [Noeud(n, couts, non_affectations)]
    nb_noeuds_ouverts = 1
    while len(OUVERT) > 0:
        noeud = heappop(OUVERT)
        if noeud.estBut():
            break
        for heritier in noeud.developpe(couts):
            heappush(OUVERT, heritier)
            nb_noeuds_ouverts += 1
    return nb_noeuds_ouverts

Pour un certain nombre de table de couts nous allons faire la comparaison du nombre de noeuds développés pour chacune des méthodes de calcul heuristique.

In [ ]:
for n in range(5, 11):
    couts = np.random.randint(1000, size=(n, n))
    nb_noeuds_total = tailleGRP(n)
    print("n = ", n)
    for heuristique in ['Somme des min des lignes', 'Somme des min des colonnes', 'Max des deux']:
        set_heuristique(heuristique)
        nb_noeuds = nb_noeuds_developpes(couts)
        if heuristique == 'Max des deux': #condition rajoutée pour des raisons d'affichage
            print(heuristique, ",\t\t\tNoeuds développés : ", nb_noeuds,
                 ",\tNoeuds développés / Noeuds total : ", '%.8f'%(nb_noeuds/nb_noeuds_total))
        else:
            print(heuristique, ",\tNoeuds développés : ", nb_noeuds,
                 ",\tNoeuds développés / Noeuds total : ", '%.8f'%(nb_noeuds/nb_noeuds_total))
    print()

On remarque donc qu'on a globalement le même dilemne en choisissant la <b style="color:#DB5B30">Somme des min des lignes</b> ou la <b style="color:#DB5B30">Somme des min des colonnes</b>, il y en a souvent un qui développe beaucoup plus de noeuds que l'autre. On constate aussi que c'est toujours l'heuristique <b style="color:#DB5B30">Max des deux</b> qui développe le moins de noeuds. On a tout intéret à choisir cette dernière !